# Boundary Value Problem 

$$
u''(x) = e^{\sin(x)}, \quad u'(0) = 0, \quad u'(1) = \alpha
$$

- Determine $\alpha$ such that the problem has at least one solution.  
- Solve the problem by finding one of its solution.

---
Integrate once:

$$
u'(x) = \int_0^x e^{\sin t} \, dt + u'(0) = \int_0^x e^{\sin t} \, dt
$$

because $u'(0) = 0$.

At $x=1$:

$$
u'(1) = \int_0^1 e^{\sin t} \, dt
$$

Thus, to have a solution, we must have:

$$
\alpha = \int_0^1 e^{\sin t} \, dt
$$

Since this integral does not have a closed-form expression, we evaluate it numerically using a dense grid and the composite Simpson method to obtain $\alpha$.

---
Integrating once more:

$$
u(x) = \int_0^x u'(s) \, ds + u(0) = \int_0^x \left( \int_0^s e^{\sin t} \, dt \right) ds + u(0)
$$

In the numerical implementation, we simply set $u(0)=0$ and compute $u(x)$ via two cumulative Simpson integrations.

In [7]:
import numpy as np

# Right-hand side
f = lambda x: np.exp(np.sin(x))

# Cumulative Simpson integration
def cum_simpson(y, x):
    n = len(x)
    if n % 2 == 0:  # make odd number of points
        x = np.append(x, x[-1] + (x[-1]-x[-2]))
        y = np.append(y, y[-1])
        n += 1
    y_int = np.zeros(n)
    for i in range(2, n, 2):
        h = x[i]-x[i-2]
        y_int[i] = y_int[i-2] + (h/6)*(y[i-2] + 4*y[i-1] + y[i])
        if i+1 < n:
            y_int[i+1] = y_int[i] + (x[i+1]-x[i]) * (y[i] + y[i+1])/2  # linear interp for next point
    return y_int[:len(x)]

# Compute alpha using very fine grid
x_dense = np.linspace(0,1,10001)
alpha = cum_simpson(f(x_dense), x_dense)[-1]
print(f"Consistency condition: alpha = {alpha:.8f}")


Consistency condition: alpha = 1.63186961


---
## Solve the problem

Once $ \alpha$ is known, we can find a solution $u(x)$:

1. Compute $u'(x) = \int_0^x f(s) ds$, satisfying $u'(0)=0$
2. Compute $u(x) = \int_0^x u'(s) ds$, with $u(0)=0$

We use cumulative Simpson's rule for both integrals.

In [2]:
# Solve BVP
def solve_bvp_simpson(N):
    x = np.linspace(0, 1, N+1)
    f_vals = f(x)

    # u'(x) using cumulative Simpson
    u_prime = cum_simpson(f_vals, x)
    # u_prime[i] = ∫₀^{x_i} e^{sin t} dt

    # u(x) using cumulative Simpson
    u = cum_simpson(u_prime, x)
    # u[i] = ∫₀^{x_i} u'(s) ds

    return x, u, u_prime

# Example solution with N=100
x, u_vals, u_prime_vals = solve_bvp_simpson(100)
print(f"u'(1) ≈ {u_prime_vals[-1]:.8f}")


u'(1) ≈ 1.63186961


---
## Convergence study

We compare solutions for increasing grid resolutions against a reference solution computed on a very fine grid.


In [6]:
# Reference solution on fine grid
x_ref, u_ref, _ = solve_bvp_simpson(1000)

Ns = [20, 40, 80, 160, 320]
results = []

for N in Ns:
    x, u_vals, _ = solve_bvp_simpson(N)
    # interpolate reference solution
    u_exact_interp = np.interp(x, x_ref, u_ref)
    err = np.abs(u_vals - u_exact_interp)
    L_inf = np.max(err)
    L2 = np.sqrt(np.mean(err**2))  
    results.append([N, L_inf, L2])

# Print convergence table
print(f"{'N':>6} | {'L_inf error':>12} | {'rate':>8} | {'L2 error':>12} | {'rate':>8}")
print("-"*60)
prev_Linf = prev_L2 = None
for N, Linf, L2 in results:
    if prev_Linf is None:
        rate_inf = rate_L2 = np.nan
    else:
        rate_inf = np.log(prev_Linf/Linf)/np.log(2)
        rate_L2  = np.log(prev_L2/L2)/np.log(2)
    print(f"{N:6d} | {Linf:12.4e} | {rate_inf:8.3f} | {L2:12.4e} | {rate_L2:8.3f}")
    prev_Linf = Linf
    prev_L2 = L2



     N |  L_inf error |     rate |     L2 error |     rate
------------------------------------------------------------
    20 |   3.4167e-03 |      nan |   3.2538e-03 |      nan
    40 |   8.4250e-04 |    2.020 |   8.2209e-04 |    1.985
    80 |   2.0836e-04 |    2.016 |   2.0586e-04 |    1.998
   160 |   5.1197e-05 |    2.025 |   5.0725e-05 |    2.021
   320 |   1.1995e-05 |    2.094 |   1.1804e-05 |    2.103
